## Tensor

TensorFlow 程序使用 tensor 数据结构来代表所有的数据, 计算图中, 操作间传递的数据都是 tensor. 你可以 把 TensorFlow tensor 看作是一个 n 维的数组或列表.

In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()
x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 使用初始化器 initializer op 的 run() 方法初始化 'x'
x.initializer.run()

# 增加一个减法 sub op, 从 'x' 减去 'a'. 运行减法 op, 输出结果
sub = tf.sub(x, a)
print sub.eval()

#sess.close()

[-2. -1.]


In [2]:
import tensorflow as tf

# Create a Constant that produces a 1x2 matrix.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', 
# represents the result of the matrix multiplication.
product = tf.matmul(matrix1, matrix2)
sess = tf.Session()
result = sess.run(product)
print(result) 

#sess.close()

[[ 12.]]


## 分布式集群
To create a TensorFlow cluster, launch a TensorFlow server on each of the machines in the cluster.
When you instantiate a Session in your client,
you pass it the network location of one of the machines in the cluster:

In [3]:
import tensorflow as tf

# with tf.Session("grpc://example.org:2222") as sess:
with tf.Session() as sess:
    # with tf.device("/cpu:0"): #cpu:1 gpu:0
    matrix1 = tf.constant([[3., 3.]])
    matrix2 = tf.constant([[2.],[2.]])
    product = tf.matmul(matrix1, matrix2)
    result = sess.run(product)
    print(result)
    sess.close()

[[ 12.]]


In [1]:
import tensorflow as tf

stat = tf.Variable(0,name="ct")

one = tf.constant(1)

nv = tf.add(stat,one)   # lazy op

up = tf.assign(stat,nv) # lazy op

init_op = tf.global_variables_initializer() # lazy op

with tf.Session() as sess:
    sess.run(init_op)
    print sess.run(stat)
    for i in range(3):
        sess.run(up)
        print  sess.run(stat)

0
1
2
3


## Fetches: 获取多个 tensor

In [5]:
import tensorflow as tf

input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)
with tf.Session() as sess:
    # result = sess.run(mul) # 获取 1个tensor
    result = sess.run([mul, intermed]) #获取 多个 tensor
    print result

[21.0, 7.0]


### Feed
feed 使用一个 tensor 值临时替换一个操作的输出结果. 可以提供 feed 数据作为 run() 调用的参数.  
feed 只在调用它的方法内有效, 方法结束, feed 就会消失.

In [6]:
inp1 = tf.placeholder(tf.float32)
inp2 = tf.placeholder(tf.float32) # dep tf.types.float32

output = tf.mul(inp1, inp2)
with tf.Session() as sess:
    print sess.run([output], feed_dict={inp1:[7.], inp2:[2.]})

[array([ 14.], dtype=float32)]


In [13]:
"""
参考：https://www.tensorflow.org/tutorials/mnist/beginners/
数据：http://www.tensorfly.cn/tfdoc/tutorials/mnist_download.html
"""
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/Users/chris/gitspace/tensor/data/MNIST_data/', one_hot=True)

x = tf.placeholder(tf.float32, [None, 784]) #None means that a dimension can be of any length

# A Variable is a modifiable tensor that lives in TensorFlow's graph of interacting operations.
# It can be used and even modified by the computation. For machine learning applications, one generally has the model parameters be Variables.
W = tf.Variable(tf.zeros([784, 10]))  
b = tf.Variable(tf.zeros([10]))

# y is our predicted probability distribution,
y = tf.nn.softmax(tf.matmul(x, W) + b)

# y′ is the true distribution (the one-hot vector with the digit labels)
y_ = tf.placeholder(tf.float32, [None, 10])

# Then we can implement the cross-entropy function, −∑y′log⁡(y):
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.3).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(200)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
# argmax给出某个tensor对象在某一维上的最大值的索引
# tf.argmax(y,1) 模型对于任一输入x预测到的标签值
# tf.argmax(y_,1) 正确的标签
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

# next_batch(100) 0.5  0.9184 0.3 0.919
# next_batch(200) 0.5 0.9204 0.3 0.9184

Extracting /Users/chris/gitspace/tensor/data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/chris/gitspace/tensor/data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/chris/gitspace/tensor/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/chris/gitspace/tensor/data/MNIST_data/t10k-labels-idx1-ubyte.gz
0.9188


In [8]:
##
"""A very simple MNIST classifier.

See extensive documentation at
http://tensorflow.org/tutorials/mnist/beginners/index.md
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

def main(_):
    # Import data
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

    # Create the model
    x = tf.placeholder(tf.float32, [None, 784])
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    y = tf.matmul(x, W) + b

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

    # The raw formulation of cross-entropy,
    #
    #   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
    #                                 reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
    # outputs of 'y', and then average across the batch.
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    # Train
    for _ in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    # Test trained model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,y_: mnist.test.labels}))

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='/Users/chris/gitspace/tensor/data/MNIST_data/',
                      help='Directory for storing input data')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)


Extracting /Users/chris/gitspace/tensor/data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/chris/gitspace/tensor/data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/chris/gitspace/tensor/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/chris/gitspace/tensor/data/MNIST_data/t10k-labels-idx1-ubyte.gz
0.9204


SystemExit: 

/Users/chris/.tensor/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
# View more python learning tutorial on my Youtube and Youku channel!!!

# Youtube video tutorial: https://www.youtube.com/channel/UCdyjiB5H8Pu7aDTNVXTTpcg
# Youku video tutorial: http://i.youku.com/pythontutorial

"""
Please note, this code is only for python 3+. If you are using python 2+, please modify the code accordingly.
"""
from __future__ import print_function
import tensorflow as tf


def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        return outputs


# define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
# add output layer
prediction = add_layer(l1, 10, 1, activation_function=None)

# the error between prediciton and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sess = tf.Session()

# tf.train.SummaryWriter soon be deprecated, use following
writer = tf.summary.FileWriter("logs/", sess.graph)

# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
sess.run(tf.global_variables_initializer())

# direct to the local dir and run this in terminal:
# $ tensorboard --logdir=logs
